In [1]:
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.animation as animation
warnings.filterwarnings('ignore')
from src.data_processing.fetch_data import fetch_data_subgraph
from src.models.liquidity_distribution import LiquidityAnalyzer, Pool, LiquidityPeriphery
pd.set_option("display.max_columns",100)
%load_ext autoreload
%autoreload 2

POOLS = [
    (3, "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "v3 USDC/ETH 0.3%", ),
    (3, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "v3 USDC/ETH 0.05%", ),
]
API_KEY = "893d7471304c5edf436c8ba60781762c"
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640".lower()
START_DATE = date(2024, 8, 1)
END_DATE = date(2024, 9, 1)
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    totalValueLockedUSD
    totalValueLockedETH
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
# POOL_ADDRESS = "0xc7bBeC68d12a0d1830360F8Ec58fA599bA1b0e9b".lower()
variables = {'pool_id': POOL_ADDRESS}
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]
pool = Pool(pool_address=POOL_ADDRESS,
            token0=data_pool['token0']['symbol'],
            token1=data_pool['token1']['symbol'],
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

analyzer = LiquidityAnalyzer(
    pool,
    start_date=START_DATE,
    end_date=END_DATE,
    threshold_dollar_volume_swaps=20000000
)

Fetched 1 records
Loading liquidity for block number 18909995
Loading liquidity for block number 18911563
Loading liquidity for block number 18912951
Loading liquidity for block number 18913979
Loading liquidity for block number 18914758
Loading liquidity for block number 18915118
Loading liquidity for block number 18915921
Loading liquidity for block number 18916179
Loading liquidity for block number 18916530
Loading liquidity for block number 18917177
Loading liquidity for block number 18917970
Loading liquidity for block number 18918719
Loading liquidity for block number 18919672
Loading liquidity for block number 18920396
Loading liquidity for block number 18920664
Loading liquidity for block number 18921197
Loading liquidity for block number 18921826
Loading liquidity for block number 18922278
Loading liquidity for block number 18923089
Loading liquidity for block number 18923952
Loading liquidity for block number 18925628
Loading liquidity for block number 18926513
Loading liquid

In [10]:
def subselect_active_liquidity(nr_tick_around):
       analyzer.df_liquidity.columns = ['block', 'active_tick', 'tick', 'liquidity', 'price', 'amount0',
              'amount1', 'amount0_real', 'amount1_real']
       liquidity = pd.merge(analyzer.sampled_blocks,
              analyzer.df_liquidity,
              left_on='block_number',
              right_on='block')
       liquidity['active_tick_adjusted'] = np.floor(liquidity['active_tick'] / pool.tick_spacing) * pool.tick_spacing
       liquidity['sb'] = liquidity['tick']-pool.tick_spacing*nr_tick_around
       liquidity['sa'] = liquidity['tick']+pool.tick_spacing*nr_tick_around
       liquidity.loc[liquidity['tick']<liquidity['active_tick_adjusted'],'amount0_real'] = 0
       liquidity.loc[liquidity['tick']>liquidity['active_tick_adjusted'],'amount1_real'] = 0
       liquidity['active_price'] = np.float64(liquidity['active_tick'].apply(lambda x: pool.adjust_price(pool.tick_to_price(x))))
       liquidity['active_price_inverted'] = 1/liquidity['active_price']
       liquidity['amount_locked'] = liquidity['amount1_real']*liquidity['active_price_inverted'] + liquidity['amount0_real']
       liquidity = liquidity.loc[liquidity['amount_locked']<50000000]
       active_liquidity = liquidity[(liquidity['active_tick_adjusted'] >= liquidity['sb']) & (liquidity['active_tick_adjusted'] <= liquidity['sa'])]
       active_liquidity = active_liquidity.reset_index(drop=True)
       return active_liquidity, liquidity

active_liquidity, liquidity_data = subselect_active_liquidity(50)
active_liquidity['amount_locked_USDC'] = active_liquidity.groupby(by='block_number')['amount_locked'].transform('sum')
active_liquidity_trading = active_liquidity[['datetime','block_number','active_tick_adjusted','active_price_inverted','cumulative_dollar_volume','amount_locked_USDC']].drop_duplicates()
active_liquidity_trading['trading_volume'] = active_liquidity_trading['cumulative_dollar_volume'] - active_liquidity_trading['cumulative_dollar_volume'].shift()
active_liquidity_trading['trading_fees'] = active_liquidity_trading['trading_volume'] * (pool.fee_tier / 1e6)
active_liquidity_trading['ROI_pool'] = active_liquidity_trading['trading_fees']/active_liquidity_trading['amount_locked_USDC']
active_liquidity_trading['date'] = active_liquidity_trading['datetime'].dt.normalize() 

In [13]:
active_liquidity.to_parquet("/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/liquidity_data.parquet")

In [47]:
example = active_liquidity.loc[active_liquidity['block_number']==20773531]

In [152]:
example.head(2)

,block_number,block_timestamp,cumulative_dollar_volume,date,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_adjusted,sb,sa,active_price,active_price_inverted,amount_locked,amount_locked_USDC
348767,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,198280,7.955235e+18,2450.414443,1.968396e+11,8.036929e+19,0.0,80.369289,198780.0,197780,198780,0.000429,2328.815375,187165.235411,1.878030e+07
348768,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,198290,7.954941e+18,2447.965376,1.967340e+11,8.040651e+19,0.0,80.406510,198780.0,197790,198790,0.000429,2328.815375,187251.916073,1.878030e+07


In [154]:
example.tick.min(), example.tick.max(), example.active_tick_adjusted.min()

(np.int32(198280), np.int32(199280), np.float64(198780.0))

In [293]:
example['amount_locked'].sum()

np.float64(18780302.406677607)

In [297]:
example['liquidity'].sum()

np.float64(7.881445837741723e+20)

In [381]:
# Example usage
print(pool.token0, pool.token1)
amount0_raw = 2328*2
amount1_raw = 1*2
current_price = 2328
lower_price = 2217
upper_price = 2450


lower_tick = 198280
upper_tick = 199280
current_tick = 198780

liquidity_calculator = LiquidityPeriphery(pool)
sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0, amount1, liquidity = liquidity_calculator.calculate_liquidity(amount0_raw, amount1_raw, lower_tick, upper_tick, current_tick)


USDC WETH


In [5]:
from src.models.backtest import LiquidityProvisionBacktester

In [11]:
initial_capital_amount_0 = 1 #ETH
initial_capital_amount_1 = 2500 #USDC
initial_capital_USD = initial_capital_amount_0*(initial_capital_amount_1/initial_capital_amount_0) + initial_capital_amount_1
tick_range = 50
rebalance_cost = 0.0001

In [7]:
backtester = LiquidityProvisionBacktester(pool, liquidity_calculator, initial_capital_amount_0, initial_capital_amount_1)

In [14]:
df_liquidity = pd.read_parquet(
        "/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/liquidity_data.parquet"
    )

In [20]:
df_liquidity.iloc[0]['active_price_inverted']

block_number                              18909995
block_timestamp                         1704080591
cumulative_dollar_volume           20028044.555969
date                                    2024-01-01
datetime                       2024-01-01 03:43:11
block                                     18909995
active_tick                                 199011
tick                                        198510
liquidity                    7536409291119563776.0
price                                  2394.700859
amount0                        184344352873.256287
amount1                     77018614993131192320.0
amount0_real                                   0.0
amount1_real                             77.018615
active_tick_adjusted                      199010.0
sb                                          198010
sa                                          199010
active_price                              0.000439
active_price_inverted                  2277.687846
amount_locked                  

In [ ]:
results_df = backtester.backtest_liquidity_provision(liquidity_data,
                                                     initial_active_liquidity, 
                                                     initial_capital_USD, 
                                                     tick_range, 
                                                     rebalance_cost)

In [ ]:
get_amounts_for_liquidity

In [ ]:

# Run the backtest
initial_capital_eth = 1
initial_capital_usdc = 2500
initial_capital = 1*2500 + 2500
tick_range = 200
rebalance_cost = 0.001

results_df = backtest_liquidity_provision(active_liquidity_trading, initial_capital, tick_range, rebalance_cost)

# Analyze results
results_df['lp_returns'] = results_df['position_capital'].pct_change()
results_df['hodl_returns'] = results_df['hodl_capital'].pct_change()

lp_cumulative_returns = (1 + results_df['lp_returns']).cumprod() - 1
hodl_cumulative_returns = (1 + results_df['hodl_returns']).cumprod() - 1

lp_sharpe_ratio = np.sqrt(365) * results_df['lp_returns'].mean() / results_df['lp_returns'].std()
hodl_sharpe_ratio = np.sqrt(365) * results_df['hodl_returns'].mean() / results_df['hodl_returns'].std()

print(f"LP Strategy Total Return: {lp_cumulative_returns.iloc[-1]:.2%}")
print(f"LP Strategy Sharpe Ratio: {lp_sharpe_ratio:.2f}")
print(f"HODL Strategy Total Return: {hodl_cumulative_returns.iloc[-1]:.2%}")
print(f"HODL Strategy Sharpe Ratio: {hodl_sharpe_ratio:.2f}")

# Plot results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results_df['datetime'], results_df['position_capital'], label='LP Strategy')
plt.plot(results_df['datetime'], results_df['hodl_capital'], label='HODL Strategy')
plt.title('Liquidity Provision vs HODL Strategy Performance')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()
plt.show()

In [87]:
def calc_active_tvl():
    current_block = liquidity.loc[liquidity['block_number']==int(blocks[0])]
    tick_t = liquidity.loc[liquidity['block_number']==int(blocks[0])]['active_tick'].iloc[0]
    tick_t_1 = liquidity.loc[liquidity['block_number']==int(blocks[1])]['active_tick'].iloc[0]
    tick_difference = abs(tick_t_1-tick_t)
    active_tick = liquidity.loc[(liquidity['block_number']==int(blocks[0]))]['active_tick_adjusted'].iloc[0]
    tvl_current_block = current_block.loc[(current_block['tick']>=(active_tick-tick_difference))&(current_block['tick']<=(active_tick+tick_difference))]['amount_locked'].sum()
    return tvl_current_block

def calculate_fees(current_block, capital):
    volume = liquidity.loc[liquidity['block_number']==current_block]['cumulative_dollar_volume'].iloc[0] + capital
    fee_rate = pool.fee_tier / 1e6
    total_fees = volume * fee_rate
    fees_earned = (capital/volume) * total_fees
    return fees_earned


tick_range = 200
rebalance_cost = 0.001
capital = 1000
for block_i in range(len(blocks)-1):
    current_block = blocks[block_i]
    next_block = blocks[block_i+1]
    current_tick = liquidity.loc[liquidity['block_number']==current_block]['active_tick_adjusted'].iloc[0]
    if block_i==0:
        t_lower = current_tick-tick_range
        t_upper = current_tick+tick_range
    # current_position = create_position(current_block, tick_range, capital)
    fees_earned = calculate_fees(current_block, capital)
    # print(current_tick, t_lower, t_upper)
    if current_tick<t_lower or current_tick>t_upper:
        #need to rebalance
        cost = capital * rebalance_cost
        capital = capital + fees_earned #- cost + 
        t_lower = current_tick-tick_range
        t_upper = current_tick+tick_range
    

199010.0 198810.0 199210.0
198970.0 198810.0 199210.0
198890.0 198810.0 199210.0
198850.0 198810.0 199210.0
198740.0 198810.0 199210.0
198730.0 198540.0 198940.0
198710.0 198540.0 198940.0
198570.0 198540.0 198940.0
198590.0 198540.0 198940.0
198530.0 198540.0 198940.0
198570.0 198330.0 198730.0
198370.0 198330.0 198730.0
198490.0 198330.0 198730.0
198610.0 198330.0 198730.0
198610.0 198330.0 198730.0
198650.0 198330.0 198730.0
198620.0 198330.0 198730.0
198670.0 198330.0 198730.0
198660.0 198330.0 198730.0
198600.0 198330.0 198730.0
198620.0 198330.0 198730.0
198740.0 198330.0 198730.0
199120.0 198540.0 198940.0
199520.0 198920.0 199320.0
199500.0 199320.0 199720.0
199380.0 199320.0 199720.0
199210.0 199320.0 199720.0
199340.0 199010.0 199410.0
199210.0 199010.0 199410.0
199240.0 199010.0 199410.0
199250.0 199010.0 199410.0
199350.0 199010.0 199410.0
199370.0 199010.0 199410.0
199220.0 199010.0 199410.0
199200.0 199010.0 199410.0
199240.0 199010.0 199410.0
199340.0 199010.0 199410.0
1

In [131]:
results_df

,block_number,datetime,price,capital,fees_earned,rebalanced
0,1098 18911563 1099 18911563 1100 1891...,1098 2024-01-01 09:00:47 1099 2024-01-01 0...,1098 4496.174045 1099 4491.680343 1100 ...,1000.776446,0.776446,False
1,2196 18912951 2197 18912951 2198 1891...,2196 2024-01-01 13:43:35 2197 2024-01-01 1...,2196 4496.174045 2197 4491.680343 2198 ...,1001.596531,0.820085,False
2,3294 18913979 3295 18913979 3296 1891...,3294 2024-01-01 17:10:35 3295 2024-01-01 1...,3294 4496.174045 3295 4491.680343 3296 ...,1002.426266,0.829735,False
3,4392 18914758 4393 18914758 4394 1891...,4392 2024-01-01 19:48:23 4393 2024-01-01 1...,4392 4496.174045 4393 4491.680343 4394 ...,1002.252579,0.826313,False
4,5490 18915118 5491 18915118 5492 1891...,5490 2024-01-01 21:00:47 5491 2024-01-01 2...,5490 4496.174045 5491 4491.680343 5492 ...,1002.937703,0.685124,False
...,...,...,...,...,...,...
3450,3789198 20770771 3789199 20770771 378920...,3789198 2024-09-17 13:57:11 3789199 2024-0...,3789198 4496.174045 3789199 4491.680343 ...,26367.715881,35.339244,False
3451,3790296 20771146 3790297 20771146 379029...,3790296 2024-09-17 15:12:11 3790297 2024-0...,3790296 4496.174045 3790297 4491.680343 ...,26402.905412,35.189531,False
3452,3791394 20771561 3791395 20771561 379139...,3791394 2024-09-17 16:36:23 3791395 2024-0...,3791394 4496.174045 3791395 4491.680343 ...,26437.786707,34.881295,False
3453,3792492 20772438 3792493 20772438 379249...,3792492 2024-09-17 19:33:11 3792493 2024-0...,3792492 4496.174045 3792493 4491.680343 ...,26472.173908,34.387202,False


In [ ]:
# Analyze results
results_df['lp_returns'] = results_df['position_capital'].pct_change()
results_df['hodl_returns'] = results_df['hodl_capital'].pct_change()

lp_cumulative_returns = (1 + results_df['lp_returns']).cumprod() - 1
hodl_cumulative_returns = (1 + results_df['hodl_returns']).cumprod() - 1

lp_sharpe_ratio = np.sqrt(365) * results_df['lp_returns'].mean() / results_df['lp_returns'].std()
hodl_sharpe_ratio

In [13]:
# Run the backtest
initial_capital = 100000


results = backtest_liquidity_position(active_liquidity_trading, initial_capital, tick_range, rebalance_cost)

# Analyze results
results['returns'] = results['capital'].pct_change()
cumulative_returns = (1 + results['returns']).cumprod() - 1
sharpe_ratio = np.sqrt(365) * results['returns'].mean() / results['returns'].std()

print(f"Total Return: {cumulative_returns.iloc[-1]:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(results['datetime'], results['capital'])
plt.title('Liquidity Provision Strategy Performance')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.show()

,block_number,block_timestamp,cumulative_dollar_volume,date,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_adjusted,sb,sa,active_price,active_price_inverted,amount_locked
0,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,192210,2.658259e+17,4496.174045,8.909607e+09,1.982588e+18,0.000000,1.982588,199010.0,192180,192240,0.000439,2277.687846,4515.717620
1,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,192220,2.658259e+17,4491.680343,8.905154e+09,1.983580e+18,0.000000,1.983580,199010.0,192190,192250,0.000439,2277.687846,4517.975931
2,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,192230,2.658259e+17,4487.191132,8.900702e+09,1.984572e+18,0.000000,1.984572,199010.0,192200,192260,0.000439,2277.687846,4520.235371
3,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,192240,2.658259e+17,4482.706408,8.896253e+09,1.985564e+18,0.000000,1.985564,199010.0,192210,192270,0.000439,2277.687846,4522.495940
4,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,192250,2.658259e+17,4478.226166,8.891807e+09,1.986557e+18,0.000000,1.986557,199010.0,192220,192280,0.000439,2277.687846,4524.757641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794683,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,203140,5.418919e+17,1507.241968,1.051584e+10,6.980363e+18,10515.837685,0.000000,198780.0,203110,203170,0.000429,2328.815375,10515.837685
3794684,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,203150,5.418919e+17,1505.735554,1.051058e+10,6.983854e+18,10510.581343,0.000000,198780.0,203120,203180,0.000429,2328.815375,10510.581343
3794685,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,203160,5.418870e+17,1504.230647,1.050523e+10,6.987284e+18,10505.233408,0.000000,198780.0,203130,203190,0.000429,2328.815375,10505.233408
3794686,20773531,1726614827,7.016129e+10,2024-09-17,2024-09-17 23:13:47,20773531,198789,203170,5.418821e+17,1502.727243,1.049989e+10,6.990714e+18,10499.885802,0.000000,198780.0,203140,203200,0.000429,2328.815375,10499.885802


In [11]:


# Assuming your data is in a DataFrame called 'liquidity'

# 1. Sample blocks
n = 20  # Choose every nth block, adjust as needed
sampled_blocks = liquidity['block_number'].unique()[::n]

# 2. Prepare the data
def prepare_data(block):
    block_data = liquidity[liquidity['block_number'] == block]
    # Create price bins
    n_bins = 150
    price_range = block_data['price'].max() - block_data['price'].min()
    bin_width = price_range / n_bins
    block_data['price_bin'] = pd.cut(block_data['price'], bins=n_bins)
    # Sum amount_locked for each bin
    liquidity_distribution = block_data.groupby('price_bin')['amount_locked'].sum().reset_index()
    liquidity_distribution['price'] = liquidity_distribution['price_bin'].apply(lambda x: x.mid)
    return block_data['datetime'].iloc[0], liquidity_distribution, block_data['active_price_inverted'].iloc[0], bin_width

# Find the maximum y-value across all blocks
max_y = 0
for block in sampled_blocks:
    _, liquidity_distribution, _, _ = prepare_data(block)
    max_y = max(max_y, liquidity_distribution['amount_locked'].max())

# 3. Create the animation
fig, ax = plt.subplots(figsize=(20, 10))

def animate(block):
    ax.clear()
    datetime, liquidity_distribution, active_price, bin_width = prepare_data(block)
    
    # Plot bar chart
    ax.bar(liquidity_distribution['price'], liquidity_distribution['amount_locked'],
           width=bin_width, alpha=0.7)
    
    # Plot active price line
    ax.axvline(x=active_price, color='red', linestyle='--', linewidth=2)
    
    ax.set_title(f'Liquidity Distribution at Block {block} at {datetime}')
    ax.set_xlabel('Price')
    ax.set_ylabel('Amount Locked')
    ax.text(0.95, 0.95, f'Active Price: {active_price:.2f}',
            transform=ax.transAxes, ha='right', va='top')
    
    # Set fixed y-axis limits
    ax.set_ylim(0, max_y * 1.1)  # Add 10% padding to the top

anim = animation.FuncAnimation(fig, animate, frames=sampled_blocks, interval=200)

# Save as gif or video
anim.save('liquidity_distribution_evolution.gif', writer='pillow')
plt.close()

In [ ]:
def calculate_position_liquidity_share(position, active_liquidity):
    shares = []
    for _, block in active_liquidity.groupby('block_number'):
        active_range_liquidity = block['amount_locked'].sum()
        active_tick = block['active_tick_adjusted'].iloc[0]
        position_liquidity = block.loc[block['active_tick_adjusted']==block['tick']]['amount_locked'].sum()
        price = 1/pool.adjust_price(pool.tick_to_price(current_tick))
        if position['lower_tick'] <= active_tick <= position['upper_tick']:
            # Position is in range
            total_value = position['initial_amount0'] * price + position['initial_amount1']
            amount0 = total_value / (2 * price)
            amount1 = total_value / 2
        elif active_tick < position['lower_tick']:
            # All in token1
            amount0 = 0
            amount1 = position['initial_amount0'] * price + position['initial_amount1']
        else:
            # All in token0
            amount0 = position['initial_amount0'] + position['initial_amount1'] / price
            amount1 = 0
            shares.append({
            'block_number': block['block_number'].iloc[0],
            'active_price_inverted': block['active_price_inverted'].iloc[0],
            'active_tick': active_tick,
            'active_range_liquidity': active_range_liquidity,
            'position_liquidity': position_liquidity,
            'amount0': amount0,
            'amount1': amount1,
            'price': price
            })
    return pd.DataFrame(shares)
shares = calculate_position_liquidity_share(position, active_liquidity)

In [152]:
def analyze_position(position, active_liquidity):
    results = []
    cumulative_fees = 0
    cumulative_fees_earned = 0
    previous_volume = 0
    position_value = np.float64(position['initial_amount0']) * np.float64(active_liquidity['price'].iloc[0]) + np.float64(position['initial_amount1']) 
    for _, block in active_liquidity.groupby('block_number'):
        
        active_tick = block['active_tick_adjusted'].iloc[0]
        price = block['active_price_inverted'].iloc[0]
        position_active = position['lower_tick'] <= active_tick <= position['upper_tick']
        active_range_liquidity = block['amount_locked'].sum()
        position_liquidity = block.loc[block['active_tick_adjusted']==block['tick']]['amount_locked'].sum()

        if position_active:
            liquidity_share = position_value / position_liquidity if position_liquidity > 0 else 0
        else:
            position_liquidity = 0
            liquidity_share = 0

        # Calculate trading volume and fees
        volume = block['cumulative_dollar_volume'].iloc[0] - previous_volume
        fees = volume * (pool.fee_tier / 1e6)
        fees_earned = fees * liquidity_share
        cumulative_fees += fees
        cumulative_fees_earned += fees_earned
        previous_volume = block['cumulative_dollar_volume'].iloc[0]

        # Update position amounts
        if position_active:
            total_value = np.float64(position['initial_amount0']) * np.float64(price) + np.float64(position['initial_amount1'])
            amount0 = total_value / (2 * price)
            amount1 = total_value / 2
        elif active_tick < position['lower_tick']:
            amount0 = 0
            amount1 = np.float64(position['initial_amount0']) * np.float64(price) + np.float64(position['initial_amount1'])
            
        else:
            amount0 = np.float64(position['initial_amount0']) + np.float64(position['initial_amount1']) / np.float64(price)
            amount1 = 0

        # Calculate position value and HODL value
        
        hodl_value = np.float64(position['initial_amount0']) * np.float64(price) + np.float64(position['initial_amount1'])
        position_value = np.float64(amount0) * np.float64(price) + np.float64(amount1) + np.float64(cumulative_fees_earned)
        # Calculate impermanent loss
        impermanent_loss = (position_value - hodl_value) / hodl_value

        results.append({
            'block_number': block['block_number'].iloc[0],
            'datetime': block['datetime'].iloc[0],
            'active_tick': active_tick,
            'price': price,
            'total_liquidity': position_liquidity,
            'position_liquidity': position_liquidity,
            'liquidity_share': liquidity_share,
            'trading_volume': volume,
            'fees': fees,
            'cumulative_fees': cumulative_fees,
            'fees_earned': fees_earned,
            'cumulative_fees_earned': cumulative_fees_earned,
            'amount0': amount0,
            'amount1': amount1,
            'position_value': position_value,
            'hodl_value': hodl_value,
            'impermanent_loss': impermanent_loss
        })

    return pd.DataFrame(results)

In [153]:
# Example usage
current_tick = active_liquidity['active_tick'].iloc[0]
tick_range = 100
position = define_position(
    current_tick - tick_range,
    current_tick + tick_range,
    1,  # 1 ETH
    1/pool.adjust_price(pool.tick_to_price(current_tick)) #equivalent in USDC
)

In [155]:
active_liquidity_range = analyze_position(position, active_liquidity)

In [156]:
active_liquidity_range['in_range'] = (position['lower_tick'] <= active_liquidity_range['active_tick']) & (active_liquidity_range['active_tick'] <= position['upper_tick'])

In [109]:
active_liquidity[['block_number','active_tick','cumulative_dollar_volume']].drop_duplicates()

,block_number,active_tick,cumulative_dollar_volume
0,20652622,198119,1.011016e+07
7,20652812,198150,2.013316e+07
14,20653267,198145,3.015785e+07
21,20653641,198198,4.020015e+07
28,20653816,198202,5.020428e+07
...,...,...,...
1855,20739768,198701,2.674862e+09
1862,20740119,198735,2.684894e+09
1869,20741107,198671,2.694894e+09
1876,20742033,198697,2.704939e+09
